In [ ]:
# ============================================
# 1. 패키지 설치
# ============================================
!pip -q install langchain langchain-openai langchain-community duckduckgo-search gradio


In [ ]:
# ============================================
# 2. 환경 변수
# ============================================
import os

# OpenAI API 키 설정 (사용자가 입력해야 함)
# from google.colab import userdata
# api_key=userdata.get('api_key')
# os.environ["OPENAI_API_KEY"] = api_key
# api_key2=userdata.get('api_key2')
# os.environ["LANGCHAIN_API_KEY"] = api_key2

from dotenv import load_dotenv

load_dotenv()
# OpenAI API 클라이언트 생성
OPENAPI_KEY = os.getenv("OPENAI_API_KEY")
LangSmith_KEY = os.getenv("LANGCHAIN_API_KEY")

# 2) LangSmith 연동 필수 환경변수
os.environ["LANGCHAIN_TRACING_V2"] = "true"      # 트레이싱 활성화
os.environ["LANGSMITH_ENDPOINT"]   = "https://api.smith.langchain.com"  # 기본값
os.environ["LANGSMITH_PROJECT"]    = "Two_Agent"                 # 수업용 프로젝트명

In [3]:
# ============================================
# 3. 모듈 임포트
# ============================================
from langchain_openai import ChatOpenAI
from langchain.tools import Tool
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from duckduckgo_search import DDGS
import gradio as gr

In [4]:
# ============================================
# 4. 검색 Tool 정의
# ============================================
def search_news(query: str) -> str:
    """DuckDuckGo 뉴스 검색"""
    with DDGS() as ddgs:
        results = list(ddgs.news(query, max_results=3))
    if not results:
        return "검색 결과 없음"
    return "\n".join([f"{r['title']} - {r['body']}" for r in results])

search_tool = Tool(
    name="search_news",
    description="주어진 주제와 관련된 최근 뉴스를 검색한다.",
    func=search_news
)

# ============================================
# 5. 에이전트 프롬프트
# ============================================
def make_agent(role: str):
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)
    prompt = ChatPromptTemplate.from_messages([
        ("system", f"너는 토론 참가자이다. 역할: {role}."),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ])
    agent = create_openai_tools_agent(llm, [search_tool], prompt)
    return AgentExecutor(agent=agent, tools=[search_tool], verbose=False)

# 두 명의 토론자 생성
agent_A = make_agent("찬성 측")
agent_B = make_agent("반대 측")

# ============================================
# 6. 토론 시뮬레이션
# ============================================
def run_debate(topic: str, rounds: int = 3):
    """두 에이전트가 교대로 토론"""
    history = []
    statement_A = f"오늘 토론 주제는 '{topic}'입니다. 나는 찬성 측으로 발언을 시작하겠습니다."
    statement_B = f"나는 반대 측으로 발언하겠습니다."

    debate_log = []
    debate_log.append(("찬성측", statement_A))
    debate_log.append(("반대측", statement_B))

    last_msg = topic
    for i in range(rounds):
        # A 발언
        res_A = agent_A.invoke({"input": last_msg, "chat_history": []})
        reply_A = res_A["output"]
        debate_log.append(("찬성측", reply_A))

        # B 반박
        res_B = agent_B.invoke({"input": reply_A, "chat_history": []})
        reply_B = res_B["output"]
        debate_log.append(("반대측", reply_B))

        last_msg = reply_B

    return debate_log

# ============================================
# 7. Gradio UI
# ============================================
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🗣️ 토론 에이전트 시뮬레이터\n주제를 입력하면 두 AI가 토론합니다.")

    topic_box = gr.Textbox(label="토론 주제", placeholder="예: 인공지능은 일자리에 위협이 될까?")
    rounds_slider = gr.Slider(1, 5, value=3, step=1, label="토론 라운드 수")
    output_chat = gr.Chatbot(label="토론 대화", height=400)
    run_btn = gr.Button("토론 시작")

    def start_debate(topic, rounds):
        log = run_debate(topic, rounds)
        return [(f"{speaker}:", msg) for speaker, msg in log]

    run_btn.click(start_debate, [topic_box, rounds_slider], output_chat)

# ============================================
# 8. 실행
# ============================================
if __name__ == "__main__":
    demo.launch(share=True, debug=True)

/tmp/ipython-input-3401175143.py:73: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  output_chat = gr.Chatbot(label="토론 대화", height=400)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://5e3eae46512f96ca94.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://5e3eae46512f96ca94.gradio.live
